In [ ]:
import os
import fitz  # PyMuPDF
from pathlib import Path

def convert_pdf_to_txt(src_folder, dst_folder):
    # 确保目标文件夹存在，如果不存在则创建
    Path(dst_folder).mkdir(parents=True, exist_ok=True)

    # 获取源文件夹下的所有 PDF 文件
    pdf_files = [f for f in os.listdir(src_folder) if f.endswith('.pdf')]

    # 遍历所有 PDF 文件
    for pdf_file in pdf_files:
        # 构建完整的源文件路径
        src_path = os.path.join(src_folder, pdf_file)
        
        # 构建目标文件路径，文件名保持不变，扩展名为 .txt
        txt_file = os.path.splitext(pdf_file)[0] + '.txt'
        dst_path = os.path.join(dst_folder, txt_file)

        # 打开 PDF 文件
        pdf_document = fitz.open(src_path)
        
        # 提取文本内容
        text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text()

        # 将提取的文本内容写入目标 TXT 文件
        with open(dst_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
        
        print(f"Converted {pdf_file} to {txt_file}")

if __name__ == "__main__":
    # 自定义源文件夹路径和目标文件夹路径
    src_folder = "/mnt/c/Users/weiyantao/Desktop/解问题/HighSchoolMathRAG/近5年高考数学全国1-2卷（已过滤图片题）"  # 替换为你的源文件夹路径
    dst_folder = "/mnt/c/Users/weiyantao/Desktop/解问题/HighSchoolMathRAG/近5年高考数学全国1-2卷（已过滤图片题）"  # 替换为你的目标文件夹路径

    # 调用转换函数
    convert_pdf_to_txt(src_folder, dst_folder)


In [1]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS
from llama_index.embeddings import LangchainEmbedding
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

import gradio as gr
from functools import partial
from operator import itemgetter
import os


In [2]:
import os
nvidia_api_key = "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"
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

In [3]:
llm = ChatNVIDIA(model="ai-nemotron-4-340b-instruct")
result = llm.invoke("what is nemo")
print(result.content)

Nemo is a character from the animated film "Finding Nemo," produced by Pixar Animation Studios and released by Walt Disney Pictures in 2003. Nemo is a young clownfish who gets separated from his father, Marlin, and ends up in a fish tank in a dentist's office in Sydney, Australia. The movie follows Marlin's journey to find and rescue Nemo, with the help of a forgetful blue tang fish named Dory. Nemo is a curious, adventurous, and determined character who learns important lessons about trust, independence, and the importance of family throughout the film.


In [4]:
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4")
embedder.embed_query("test")

[-0.0193023681640625,
 -0.0175018310546875,
 -0.03448486328125,
 -0.00595855712890625,
 0.03192138671875,
 0.046142578125,
 0.0007371902465820312,
 -0.01788330078125,
 0.0220184326171875,
 0.004657745361328125,
 0.087158203125,
 0.0168914794921875,
 0.02239990234375,
 -0.039398193359375,
 0.0133819580078125,
 0.02166748046875,
 -0.0005540847778320312,
 -0.019439697265625,
 -0.0280303955078125,
 0.0257110595703125,
 0.0145416259765625,
 0.001117706298828125,
 0.051910400390625,
 -0.0037689208984375,
 0.017974853515625,
 0.0166778564453125,
 -0.01446533203125,
 -0.034820556640625,
 0.03497314453125,
 0.061920166015625,
 -0.03436279296875,
 -0.0911865234375,
 0.0003790855407714844,
 0.00490570068359375,
 0.01529693603515625,
 -0.04266357421875,
 0.0423583984375,
 0.01262664794921875,
 -0.034820556640625,
 0.0012845993041992188,
 0.009063720703125,
 -0.0202789306640625,
 -0.002628326416015625,
 0.01959228515625,
 -0.022857666015625,
 0.0033321380615234375,
 -0.00600433349609375,
 -0.007289

In [5]:
import os
from tqdm import tqdm
from pathlib import Path

#读取并预处理文本数据，存储在 documents 和 sources 列表中

src_folder = "/mnt/c/Users/weiyantao/Desktop/解问题/HighSchoolMathRAG/近5年高考数学全国1-2卷（已过滤图片题）"
ps = os.listdir(src_folder)
data = []
sources = []
docs_name = []
for p in ps:
    if p.endswith('.txt'):
        path2file = os.path.join(src_folder, p)
        docs_name.append(path2file)
        with open(path2file, encoding="utf-8") as f:
            lines = f.readlines()
            for line in lines:
                if len(line) >= 1:
                    data.append(line)
                    sources.append(path2file)

documents = [d for d in data if d != '\n']
print(len(data), len(documents), data[0])


34441 34441 2020 年全国统一高考数学试卷（理科）（新课标Ⅰ）



In [6]:
from operator import itemgetter
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import CharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import faiss
# create my own uuid
text_splitter = CharacterTextSplitter(chunk_size=400, separator=" ")
docs = []
metadatas = []


In [7]:
for i, d in enumerate(documents):
    splits = text_splitter.split_text(d)
    #print(len(splits))
    docs.extend(splits)
    metadatas.extend([{"source": sources[i]}] * len(splits))
### 将创建好的embed存储到本地
store = FAISS.from_texts(docs, embedder , metadatas=metadatas)
store.save_local('./embed')

In [8]:
### 从本地读取已经创建好的embed
vecstores = [FAISS.load_local(folder_path="/mnt/c/Users/weiyantao/Desktop/解问题/HighSchoolMathRAG/test/embed", embeddings=embedder)]

In [9]:
embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## 初始化一个空的 FAISS 索引并将其他索引合并到其中
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

if 'docstore' not in globals():
    docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 34441 chunks


In [10]:
llm = ChatNVIDIA(model="ai-nemotron-4-340b-instruct") | StrOutputParser()
convstore = default_FAISS()

In [ ]:
doc_names_string = "\n"
for doc_name in docs_name:
    doc_names_string += doc_name+"\n"
    
def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_names_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.Reply must more than 100 words)"
), ('user', '{input}')])

## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## 将较长的文档重新排序到输出文本的中心， RunnableLambda在链中运行无参自定义函数 ，长上下文重排序（LongContextReorder）
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)
stream_chain = chat_prompt | llm

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ##首先根据输入的消息进行检索
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## 然后流式传输stream_chain的结果
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## 优化信息打印的格式
        if not return_buffer:
            line_buffer += token
            if "\n" in line_buffer:
                line_buffer = ""
            if ((len(line_buffer)>84 and token and token[0] == " ") or len(line_buffer)>100):
                line_buffer = ""
                yield "\n"
                token = "  " + token.lstrip()
        yield buffer if return_buffer else token

    ##最后将聊天内容保存到对话内存缓冲区中
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)